# Chains in LangChain

## What are Chains?

**Chains** are sequences of calls that link multiple components together to create a complete workflow. A chain typically combines a **Prompt Template**, a **Model (LLM)**, and an **Output Parser** into a unified pipeline.

### Why Use Chains?

1. **Composability**: Build complex applications by connecting independent components
2. **Modularity**: Each component (prompt, model, parser) can be tested separately
3. **Reusability**: Create once, use multiple times with different inputs
4. **LCEL Integration**: Use the modern pipe operator `|` for elegant, readable code
5. **Flexibility**: Easy to modify, extend, or rearrange components
6. **Error Handling**: Chain failures can be caught and handled systematically

### Chains in the LangChain Ecosystem

```
Input → Prompt Template → Model → Output Parser → Output
        ↑___________ Chain Pipeline ___________↑
```

Chains orchestrate the flow of data through your LangChain application, transforming raw inputs into processed outputs.

### Key Principle

**Chains are the "glue" that holds LangChain components together**. They enable LCEL (LangChain Expression Language) syntax: `prompt | model | parser`, making your code clean and composable.

## Chain Types

| Type | Purpose | Complexity |
|------|---------|-----------|
| **Simple Chain** | Single linear flow: prompt → model → parser | Beginner |
| **Sequential Chain** | Multiple steps where output feeds to next input | Intermediate |
| **Parallel Chain** | Multiple independent branches running simultaneously | Intermediate |
| **Conditional Chain** | Branching logic based on conditions | Advanced |
| **Custom Chain** | Complex data flow with custom logic (RunnablePassthrough, RunnableLambda) | Advanced |


In [ ]:
# Imports 
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

# Model
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

# Parser
parser = StrOutputParser()

# Prompt
template = PromptTemplate(
    template='Generate some intresting facts about {topic} in 5 lines.',
    input_variables=['topic']
)

# Chain
chain = template | model | parser
result = chain.invoke({'topic': 'Agentic AI'})
print(result)

## Simple Chain (LCEL)

### Definition

A **Simple Chain** is the most basic type of chain, combining exactly three components in a linear sequence: Prompt Template → Model → Output Parser.

### Key Features

- **Linear Flow**: Single path from input to output
- **LCEL Syntax**: Uses the pipe operator `|` for clean, readable code
- **Automatic Passing**: Output of each step automatically becomes input to the next
- **Type Safe**: Each component handles type conversions
- **Fast Execution**: Minimal overhead for straightforward workflows

### Use Cases

- Single-turn question answering
- Text summarization
- Content generation with specific formatting
- Basic text transformation
- Simple classification tasks

### Best Practices

✅ **DO:**
- Keep prompts clear and specific
- Use appropriate output parsers (StrOutputParser for strings, JsonOutputParser for structured data)
- Test each component independently before chaining
- Add error handling for production use

❌ **DON'T:**
- Chain too many steps without testing intermediate outputs
- Use complex prompts without clear instructions
- Forget to specify output parser
- Hardcode values that should be template variables

### Basic Syntax

```python
chain = prompt_template | model | output_parser
result = chain.invoke({'variable': 'value'})
```


### Sequential Chain

In [ ]:
# Imports 
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

# Model
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    task="text-generation",
    temperature=0.3)
model = ChatHuggingFace(llm=llm)

# Parser
parser = StrOutputParser()

# Prompt
prompt1 = PromptTemplate(
    template='Generate a Detailed concised report about {topic}.',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarise a report in concised short points covering only important things.\n Report:{topic}',
    input_variables=['topic']
)

chain = prompt1 | model | parser | prompt2 | model | parser
result = chain.invoke({'topic':'Agentic AI'})

print(result)

### Code Explanation: Simple Chain

1. **Import Components**: `PromptTemplate`, `HuggingFaceEndpoint`, `ChatHuggingFace`, `StrOutputParser`
2. **Initialize Model**: Set up LLM with specific repo_id and temperature
3. **Create Prompt Template**: Define template with variables to be filled
4. **Create Parser**: Use `StrOutputParser` to extract text from model response
5. **Build Chain**: Use pipe operator `|` to connect components
6. **Invoke Chain**: Pass input dictionary to trigger execution
7. **Get Result**: Access the final processed output

### Key Points

- **The Pipe Operator (`|`)**: Takes output from left side and feeds as input to right side
  - `prompt | model | parser` executes: 
    - Input dict → Prompt (formats) → String
    - String → Model (generates) → AIMessage
    - AIMessage → Parser (extracts text) → Clean String
- **LCEL Advantage**: No manual variable passing needed; automatic type conversion
- **Scalability**: Easy to add more steps: `prompt | model | parser | next_prompt | model | parser`


## Sequential Chain

### Definition

A **Sequential Chain** links multiple chains together where the **output of one chain becomes the input of the next chain**. This enables multi-step workflows like generating content then summarizing it.

### Key Features

- **Multi-Step Processing**: Execute multiple transformations sequentially
- **Data Flow Continuity**: Output automatically feeds into next step's input
- **Progressive Refinement**: Each step can modify or enhance the data
- **Automatic Variable Mapping**: No manual variable assignment needed
- **LCEL Compatible**: Built with pipe operator syntax

### Use Cases

- Generate detailed content → Summarize it
- Brainstorm ideas → Evaluate and rank them
- Extract information → Format and structure it
- Draft text → Polish and review → Finalize
- Multi-stage data processing pipelines

### Advantages Over Simple Chains

| Aspect | Simple Chain | Sequential Chain |
|--------|--------------|------------------|
| **Steps** | 1 (prompt → model → parser) | Multiple (chain → chain → ...) |
| **Output Reuse** | Not directly | Each step output becomes input |
| **Use Case** | Single transformation | Multi-stage workflows |
| **Complexity** | Low | Medium |


In [ ]:
## Parallel Chain

### Definition

A **Parallel Chain** executes multiple independent chains simultaneously based on the **same input**. Each branch runs independently, and their results are combined into a single output dictionary.

### Key Features

- **Simultaneous Execution**: Multiple chains run at the same time
- **Independent Branches**: Each chain processes the same input separately
- **Merged Results**: Dictionary containing all parallel outputs
- **Efficient Processing**: Faster than sequential chains for independent tasks
- **RunnableParallel**: LangChain's component for parallel execution

### Use Cases

- Generate multiple perspectives on a topic
- Create summary + key points + quiz from one input
- Parallel sentiment analysis and entity extraction
- Multi-model comparisons (run with different models)
- Content generation with different tones/styles
- Research paper: abstract + methodology + results from same topic

### Advantages

| Aspect | Sequential Chain | Parallel Chain |
|--------|------------------|-----------------|
| **Speed** | Slower (step by step) | Faster (simultaneous) |
| **Dependencies** | Each step depends on previous | Independent steps |
| **Use Case** | Pipeline processing | Multi-aspect analysis |
| **Output** | Single value | Dictionary of results |

### Parallel Chain Architecture

```
         ┌─→ Prompt1 → Model → Parser → Output1
Input ───┼─→ Prompt2 → Model → Parser → Output2  (all run simultaneously)
         └─→ Prompt3 → Model → Parser → Output3

Result: {'output1': value, 'output2': value, 'output3': value}
```


### Code Explanation: Sequential Chain

1. **First Chain**: `prompt1 | model | parser`
   - Takes `{topic}` as input
   - Generates detailed report about the topic
   - Parses output as string
   - Result: Full report text

2. **Second Chain**: `prompt2 | model | parser`
   - Takes the report from first chain
   - Automatically fills `{topic}` variable (from first chain output)
   - Summarizes the report into key points
   - Result: Concise summary

3. **Complete Pipeline**: `prompt1 | model | parser | prompt2 | model | parser`
   - All steps execute in order
   - No manual variable passing
   - Output of parser 1 → Input to prompt2
   - Final output is the summary

### How Data Flows

```
Input: {'topic': 'Agentic AI'}
         ↓
    prompt1 formats: "Generate a Detailed report about Agentic AI."
         ↓
    model generates: [Full AI explanation]
         ↓
    parser extracts: "The detailed report..."
         ↓
    prompt2 formats: "Summarize this report: The detailed report..."
         ↓
    model generates: [Summary of report]
         ↓
    parser extracts: "Key points: 1. ... 2. ..."
         ↓
    Final Output
```

### Best Practices for Sequential Chains

✅ **DO:**
- Test each component separately first
- Use clear variable names that match template placeholders
- Add intermediate parsing steps if needed
- Log outputs at each step for debugging

❌ **DON'T:**
- Skip testing individual steps
- Create overly long chains (5+ steps) without breaks
- Use incompatible prompts (prompt output format ≠ next prompt input format)


In [ ]:
# imports
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

# Models
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.2",
    task="text-generation")
model = ChatHuggingFace(llm = llm)

llm2 = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    temperature=0.7)
model2 = ChatHuggingFace(llm = llm2)

#parsers
parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment : Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')
# since it is a llm we dont have control on its responce so we need to structure the output using Pydantic output parser to get controled and consistent output.
parser2 = PydanticOutputParser(pydantic_object=Feedback)

# review
review = '''WI got a defective ASUS laptop — it would shut down during normal use, the screen was flickering, it would hang, and I couldn’t close apps from the taskbar. I complained to Amazon customer care within 3 days, while it was still under the replacement period. They told me to contact ASUS support.

I spoke with ASUS customer care, and they sent a technician. He checked and confirmed the problem, then said Windows would need to be reinstalled. I told him that this is a new product, and it shouldn’t arrive defective — I wanted a replacement, not a repair. He wrote down the issue in a letter and said he’d check with the company and get back to me.

Two days passed with no response. When I called again, they said the replacement request was cancelled. I contacted customer care again, and they sent another technician. He submitted a replacement request and took a copy of the invoice.

Now, for a replacement, a DOA (Dead on Arrival) letter is required, which is made using the details from the invoice. But my invoice didn’t have the laptop’s serial number. Because of that, ASUS refused to issue the DOA letter, saying you needed to talk to Amazon.

After talking to Amazon 5–7 times, they kept saying their policy doesn’t allow adding a serial number to the invoice. ASUS said they can’t make the DOA letter without it. Later, I gave them the warranty slip, and after 2–3 days they somehow managed to create the letter.

But when I tried to submit the replacement request on Amazon, they said replacement isn’t available at all — it won’t happen. They told me to contact the manufacturer or seller. The seller’s phone didn’t even connect. ASUS then said it’s Amazon’s issue, not theirs.

So now both of them are refusing to take responsibility, and the replacement just isn’t happening.

I recieved the defective laptop on 27 September , and today is 17 october and still nothing has changed.'''

# get detailed summary of review
prompt = PromptTemplate(
    template='''You are a customer experience analyst.

Summarize the following customer review clearly and objectively.

Guidelines:

Identify whether the review is positive, negative, or mixed

Highlight the main points the customer mentioned

Do NOT add assumptions or extra information

Keep it short and factual (2–4 bullet points or 2–3 lines)

Customer Review:
{feedback}''',
    input_variables=['feedback'])

# Get positive or negative
prompt1 = PromptTemplate(
    template=' Classify the sentiment of the following feedback text into possitive or negative:\n {feedback}. \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction': parser2.get_format_instructions()})

# Responce for positive review
prompt2 = PromptTemplate(
    template='''You are a customer support manager for a professional company.

Write a calm, polite, and empathetic response to the following negative customer review.

Guidelines:

Acknowledge the customer’s concern clearly

Apologize where appropriate (without admitting legal fault)

Offer a solution or next step

Keep the tone respectful and professional

Do not argue or sound defensive

Keep it concise but helpful

Customer Review:
{feedback}
''',
    input_variables=['feedback'])

# Responce for negative review
prompt3 = PromptTemplate(
    template='''You are a customer support manager for a professional company.

Write a warm, appreciative, and professional response to the following positive customer review.

Guidelines:

Thank the customer genuinely

Acknowledge what they liked specifically

Reinforce the company’s commitment to quality/service

Keep the tone friendly but professional

Keep it concise

Customer Review:
{feedback}
''',
    input_variables=['feedback'])

# summary of review
sentiment = prompt | model | parser
review_summary = sentiment.invoke({'feedback': review})

# classify positive or negaive
classifier_chain = prompt1 | model | parser2

# conditional runnable for positive or negative responce
branched_chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', prompt3 | model2 | parser),
    (lambda x: x.sentiment == 'negative', prompt2 | model2 | parser),
    RunnableLambda(lambda x: ' we could not find sentiment'))

# final chain for responce
chain = classifier_chain | branched_chain
result = chain.invoke({'feedback': review})

# Output
print('--> sentiment:\n',review_summary)
print()
print(result)

### Code Explanation: Conditional Chain

1. **Define Classification Prompt**:
   - `prompt1`: Classifies sentiment as positive/negative
   - Uses `PydanticOutputParser` to enforce structured output
   - Returns `Feedback` object with sentiment field

2. **Create Classifier Chain**:
   - `classifier_chain = prompt1 | model | parser2`
   - Output: Pydantic object with `.sentiment` attribute

3. **Define Response Prompts**:
   - `prompt2`: Response template for negative sentiment
   - `prompt3`: Response template for positive sentiment
   - Each tailored to specific sentiment

4. **Create RunnableBranch**:
   ```python
   branched_chain = RunnableBranch(
       (lambda x: x.sentiment == 'positive', prompt3 | model2 | parser),
       (lambda x: x.sentiment == 'negative', prompt2 | model2 | parser),
       RunnableLambda(lambda x: 'sentiment not found')
   )
   ```
   - Condition: Check `x.sentiment` value
   - If positive → Use prompt3 (appreciative tone)
   - If negative → Use prompt2 (empathetic tone)
   - Fallback: Return error message

5. **Complete Pipeline**:
   - `chain = classifier_chain | branched_chain`
   - First classify sentiment
   - Then route to appropriate response template
   - Generate customized response

### How Conditional Logic Works

```
Customer Review Input
         ↓
[Classify Sentiment] → Feedback(sentiment='negative')
         ↓
[Evaluate Condition] → x.sentiment == 'positive'? NO
                    → x.sentiment == 'negative'? YES ✓
         ↓
[Execute Negative Response Chain]
  prompt2 | model | parser
         ↓
[Generate Empathetic Response]
"We understand your frustration..."
         ↓
Final Response Output
```

### Best Practices for Conditional Chains

✅ **DO:**
- Use structured output parsers (Pydantic) for reliable classification
- Have clear, distinct conditions
- Provide meaningful fallback chains
- Test all branches thoroughly
- Use lambda for simple conditions, functions for complex

❌ **DON'T:**
- Use overlapping conditions
- Forget to handle edge cases
- Make conditions overly complex
- Skip testing fallback paths
- Create too many branches (>5)


## Summary: Chains in LangChain

### Chain Types Quick Reference

| Chain Type | Pattern | Best For | Complexity |
|-----------|---------|----------|-----------|
| **Simple** | `prompt \| model \| parser` | Single transformation | Low |
| **Sequential** | `chain1 \| chain2 \| chain3` | Multi-step workflows | Medium |
| **Parallel** | `RunnableParallel({...})` | Multi-aspect analysis | Medium |
| **Conditional** | `RunnableBranch(...)` | Dynamic routing | Medium-High |

### Key Takeaways

1. **LCEL is Powerful**: The pipe operator `|` makes composition simple and readable
2. **Sequential**: Output automatically becomes input (variable mapping)
3. **Parallel**: Run independent tasks simultaneously for speed
4. **Conditional**: Route execution based on data properties
5. **Composability**: Combine all types for complex workflows

### When to Use Each Chain Type

| Requirement | Chain Type |
|------------|-----------|
| Generate text → Parse it | Simple Chain |
| Generate → Summarize → Polish | Sequential Chain |
| Get summary + notes + quiz | Parallel Chain |
| Different response per sentiment | Conditional Chain |
| All of the above | Combined approach |

### Common Patterns

```python
# Pattern 1: Simple chain
chain = prompt | model | parser

# Pattern 2: Sequential chain
chain = (prompt1 | model | parser) | (prompt2 | model | parser)

# Pattern 3: Parallel chain
parallel = RunnableParallel({'output1': chain1, 'output2': chain2})
final_chain = parallel | merge_prompt | model | parser

# Pattern 4: Conditional chain
branched = RunnableBranch(
    (lambda x: condition(x), chain_if_true),
    default_chain
)
final_chain = classifier | branched

# Pattern 5: Complete workflow
full_chain = (prompt | model | parser) | parallel | branched
```

### Error Handling Tips

✅ **DO:**
- Test each component independently
- Add validation prompts
- Use try-except in invoke calls
- Log intermediate outputs
- Handle timeouts and API errors

### Performance Optimization

- Use parallel chains for independent tasks
- Set appropriate temperature values
- Cache prompt templates if reused
- Use faster models for simple tasks
- Monitor API rate limits

### Next Steps

After mastering chains:
1. **Memory**: Add conversation history to chains
2. **Agents**: Create autonomous decision-making systems
3. **RAG**: Combine chains with document retrieval
4. **Custom Runnables**: Build specialized components
5. **Production**: Deploy chains with error handling and monitoring
